In [ ]:
import sqlite3
import pandas as pd
import re

In [ ]:
# Conexão no Banco
con = sqlite3.connect("database/documentos.db")

try:
    query = """
         select * from base
        """
    df = pd.read_sql_query(query, con)
except sqlite3.Error as e:
    print(f"Erro ao inserir no banco de dados: {e}")
finally:
    con.close()

In [ ]:
def clean_text(text: str):
    """
    Limpa alguns padrões indesejados do texto

    Parameters:
    ----------
    text : str
        Nome do arquivo PDF processado.
    
    Returns:
    -------
    str
        Um único texto após o processamento
    """
    # Remove caracteres não alfanuméricos indesejados, mantendo pontuações básicas
    text = re.sub(r'[^\w\s,.?!]', '', text)
    # Remove espaços extras
    text = re.sub(r'\s+', ' ', text)
    # Remove espaços extras antes de pontuação
    text = re.sub(r'\s+([,.?!])', r'\1', text)
    
    # Remover sublinhados excessivos
    text = re.sub(r'_{2,}', ' ', text)  # Substituir dois ou mais '_' por um espaço
    
    # 1. Remover caracteres estranhos e não alfabéticos repetidos
    text = re.sub(r'[^\w\s,.ºª/()\'"-]', ' ', text)  # Permitir apenas caracteres específicos
    
    # 2. Corrigir formatação de parágrafos e pontuações
    text = re.sub(r'\.(?=\S)', '. ', text)  # Adicionar espaço após pontos finais
    text = re.sub(r',(?=\S)', ', ', text)  # Adicionar espaço após vírgulas
    text = re.sub(r' +', ' ', text).strip()  # Garantir que espaços em excesso sejam eliminados

    # 3. Normalizar para letras maiúsculas/minúsculas onde apropriado
    text = text.title()  # Opcional: transformar em título (primeiras letras maiúsculas)
    
    return text.strip()

In [ ]:
for index, row in df.iterrows():
    textoo = clean_text(row['ocr_text_unida'])
    df.at[index, 'tratado_ocr_text_unida'] = textoo
    
    if len(textoo) < 20:
        df.drop(index=index, inplace=True)

In [ ]:
# Criação de outro banco com o texto tratado

# Criação do banco SQLite
conn = sqlite3.connect("database/documentos_tratados.db")
cursor = conn.cursor()

# Salvando o dataframe como uma tabela no banco
df.to_sql("base", conn, if_exists="replace", index=False)

# Confirmar e fechar conexão
conn.commit()
conn.close()